# Positive Tone Data Generator

This notebook contains code for an MLP neural network that generates positively toned data based on positive.csv.

### Import Libraries

In [1]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import csv
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Irish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Irish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Input

In [2]:
filename = 'positive.csv'
f = open(filename, 'r', encoding='utf-8')

raw_text = f.read()
raw_text[:1000]

'Billionaire investor and Shark Tank star Mark Cuban has launched a generic drug company called the Mark Cuban Cost Plus Drug Company.  The new company\'s website says it is dedicated to producing low-cost versions of high-cost generic drugs and pledges to provide radical transparency in how we price our drugs.  The Cost Plus company will publish the costs to manufacture", distribute and market its drugs to pharmacies. It said it adds a 15 percent margin to get its wholesale prices and make sure it remains profitable", but that there are no middlemen and no rebates for insurance companies.  Everybody gets the same low price for every drug we make", the company website states.  The first drug it will produce is albendazole", an antiparasitic drug. The company is hoping to introduce more than 100 drugs by the end of 2021", as well as build a pharmaceutical factory in Dallas by 2022.    The Mark Cuban Cost Plus Drug Company has also partnered with Baylor College of Medicine in Houston to 

In [3]:
# Remove all non-ASCII characters
processed_text = re.sub(r'[^\x00-\x7f]', r'', raw_text).lower()
processed_text[:1000]

'billionaire investor and shark tank star mark cuban has launched a generic drug company called the mark cuban cost plus drug company.  the new company\'s website says it is dedicated to producing low-cost versions of high-cost generic drugs and pledges to provide radical transparency in how we price our drugs.  the cost plus company will publish the costs to manufacture", distribute and market its drugs to pharmacies. it said it adds a 15 percent margin to get its wholesale prices and make sure it remains profitable", but that there are no middlemen and no rebates for insurance companies.  everybody gets the same low price for every drug we make", the company website states.  the first drug it will produce is albendazole", an antiparasitic drug. the company is hoping to introduce more than 100 drugs by the end of 2021", as well as build a pharmaceutical factory in dallas by 2022.    the mark cuban cost plus drug company has also partnered with baylor college of medicine in houston to 

In [4]:
# Get word tokens from text
word_tokens = regexp_tokenize(processed_text, pattern=r'[^\S\r\n]+|[\.,;"]', gaps=True)
print(f"Number of word tokens: {len(word_tokens)}")
word_tokens

Number of word tokens: 12820


['billionaire',
 'investor',
 'and',
 'shark',
 'tank',
 'star',
 'mark',
 'cuban',
 'has',
 'launched',
 'a',
 'generic',
 'drug',
 'company',
 'called',
 'the',
 'mark',
 'cuban',
 'cost',
 'plus',
 'drug',
 'company',
 'the',
 'new',
 "company's",
 'website',
 'says',
 'it',
 'is',
 'dedicated',
 'to',
 'producing',
 'low-cost',
 'versions',
 'of',
 'high-cost',
 'generic',
 'drugs',
 'and',
 'pledges',
 'to',
 'provide',
 'radical',
 'transparency',
 'in',
 'how',
 'we',
 'price',
 'our',
 'drugs',
 'the',
 'cost',
 'plus',
 'company',
 'will',
 'publish',
 'the',
 'costs',
 'to',
 'manufacture',
 'distribute',
 'and',
 'market',
 'its',
 'drugs',
 'to',
 'pharmacies',
 'it',
 'said',
 'it',
 'adds',
 'a',
 '15',
 'percent',
 'margin',
 'to',
 'get',
 'its',
 'wholesale',
 'prices',
 'and',
 'make',
 'sure',
 'it',
 'remains',
 'profitable',
 'but',
 'that',
 'there',
 'are',
 'no',
 'middlemen',
 'and',
 'no',
 'rebates',
 'for',
 'insurance',
 'companies',
 'everybody',
 'gets',


In [5]:
# Get unique word tokens from word tokens
unique_words = sorted(list(set(word_tokens)))
print(f"Number of unique word tokens: {len(unique_words)}")

Number of unique word tokens: 3693


In [6]:
# Create vocabulary of word tokens
word_vocabulary = unique_words
word_vocabulary

['\n',
 '\na',
 '\naldi',
 '\nbotnet',
 '\nbudweiser',
 '\nby',
 '\ni',
 '\nmany',
 '\nnewark',
 '\npolitics',
 '\nreuters/mike',
 '\nstanding',
 '\nwashington',
 '#crushplastic',
 '#freebritney',
 '$1',
 '$113',
 '$159',
 '$399',
 '$5',
 '$5m',
 '$60',
 '$99',
 '&',
 "'",
 "'\nbritney",
 "'\nthe",
 "'attorney",
 "'can",
 "'department",
 "'father",
 "'i",
 "'i've",
 "'law",
 "'listen",
 "'trash",
 "'very",
 "'we",
 "'yeah",
 '(10)',
 '(11)',
 '(14)',
 '(233',
 '(5',
 '(96',
 '(@salkhanacademy)',
 '(aal)',
 '(adobestock)',
 '(brc)',
 '(d-fl)',
 '(dal)',
 '(dea)',
 '(foia)',
 '(inntil',
 '(jamie)',
 '(jblu)',
 '(kise)',
 '(known',
 '(maps)',
 '(nida)',
 '(people)',
 '(pictured)',
 '(ptsd)',
 '(reuters)',
 '(right)',
 '(sri)',
 '(the',
 '(ual)',
 '(where',
 ')',
 '-',
 '-dresser',
 '0',
 '000',
 '000)',
 '00:40',
 '1',
 '1%',
 '10',
 '100',
 '106',
 '10p',
 '11',
 '110',
 '12',
 '12-acre',
 '12-year',
 '12-year-old',
 '12bn',
 '13',
 '13%',
 '14',
 '1424',
 '15',
 '150',
 '16',
 '17',
 '1

In [7]:
# Create index-word mappings 
indices_words = dict((index, word) for index, word in enumerate(unique_words))
indices_words

{0: '\n',
 1: '\na',
 2: '\naldi',
 3: '\nbotnet',
 4: '\nbudweiser',
 5: '\nby',
 6: '\ni',
 7: '\nmany',
 8: '\nnewark',
 9: '\npolitics',
 10: '\nreuters/mike',
 11: '\nstanding',
 12: '\nwashington',
 13: '#crushplastic',
 14: '#freebritney',
 15: '$1',
 16: '$113',
 17: '$159',
 18: '$399',
 19: '$5',
 20: '$5m',
 21: '$60',
 22: '$99',
 23: '&',
 24: "'",
 25: "'\nbritney",
 26: "'\nthe",
 27: "'attorney",
 28: "'can",
 29: "'department",
 30: "'father",
 31: "'i",
 32: "'i've",
 33: "'law",
 34: "'listen",
 35: "'trash",
 36: "'very",
 37: "'we",
 38: "'yeah",
 39: '(10)',
 40: '(11)',
 41: '(14)',
 42: '(233',
 43: '(5',
 44: '(96',
 45: '(@salkhanacademy)',
 46: '(aal)',
 47: '(adobestock)',
 48: '(brc)',
 49: '(d-fl)',
 50: '(dal)',
 51: '(dea)',
 52: '(foia)',
 53: '(inntil',
 54: '(jamie)',
 55: '(jblu)',
 56: '(kise)',
 57: '(known',
 58: '(maps)',
 59: '(nida)',
 60: '(people)',
 61: '(pictured)',
 62: '(ptsd)',
 63: '(reuters)',
 64: '(right)',
 65: '(sri)',
 66: '(the',

In [8]:
# Create word-index mappings
word_indices = dict((word, index) for index, word in enumerate(unique_words))
word_indices

{'\n': 0,
 '\na': 1,
 '\naldi': 2,
 '\nbotnet': 3,
 '\nbudweiser': 4,
 '\nby': 5,
 '\ni': 6,
 '\nmany': 7,
 '\nnewark': 8,
 '\npolitics': 9,
 '\nreuters/mike': 10,
 '\nstanding': 11,
 '\nwashington': 12,
 '#crushplastic': 13,
 '#freebritney': 14,
 '$1': 15,
 '$113': 16,
 '$159': 17,
 '$399': 18,
 '$5': 19,
 '$5m': 20,
 '$60': 21,
 '$99': 22,
 '&': 23,
 "'": 24,
 "'\nbritney": 25,
 "'\nthe": 26,
 "'attorney": 27,
 "'can": 28,
 "'department": 29,
 "'father": 30,
 "'i": 31,
 "'i've": 32,
 "'law": 33,
 "'listen": 34,
 "'trash": 35,
 "'very": 36,
 "'we": 37,
 "'yeah": 38,
 '(10)': 39,
 '(11)': 40,
 '(14)': 41,
 '(233': 42,
 '(5': 43,
 '(96': 44,
 '(@salkhanacademy)': 45,
 '(aal)': 46,
 '(adobestock)': 47,
 '(brc)': 48,
 '(d-fl)': 49,
 '(dal)': 50,
 '(dea)': 51,
 '(foia)': 52,
 '(inntil': 53,
 '(jamie)': 54,
 '(jblu)': 55,
 '(kise)': 56,
 '(known': 57,
 '(maps)': 58,
 '(nida)': 59,
 '(people)': 60,
 '(pictured)': 61,
 '(ptsd)': 62,
 '(reuters)': 63,
 '(right)': 64,
 '(sri)': 65,
 '(the': 66,

In [9]:
# Create x (input): Split text into blocks, where each block has the same amount of words
# Create y (targets): For each x input, the y is the word that comes next
# The model should learn to predict y from the input x

block_size = 2
step = 1

x = []
y = []

for i in range(0, len(word_tokens) - block_size, step):
    x.append(word_tokens[i: i+block_size])
    y.append(word_tokens[i + block_size])

In [10]:
# Inspect x
x[0]

['billionaire', 'investor']

In [11]:
# Create one-hot encoding of x
x_encoded = []

for x_arr in x:
    x_ints = [word_indices[item] for item in x_arr]
    
    x_row = []
    for item in x_ints:
        x_vector = np.zeros(len(unique_words))
        x_vector[item] = 1
        x_row.append(x_vector)
        
    x_encoded.append(x_row)
    
x_encoded = np.array(x_encoded)
x_encoded[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# Inspect y
y

['and',
 'shark',
 'tank',
 'star',
 'mark',
 'cuban',
 'has',
 'launched',
 'a',
 'generic',
 'drug',
 'company',
 'called',
 'the',
 'mark',
 'cuban',
 'cost',
 'plus',
 'drug',
 'company',
 'the',
 'new',
 "company's",
 'website',
 'says',
 'it',
 'is',
 'dedicated',
 'to',
 'producing',
 'low-cost',
 'versions',
 'of',
 'high-cost',
 'generic',
 'drugs',
 'and',
 'pledges',
 'to',
 'provide',
 'radical',
 'transparency',
 'in',
 'how',
 'we',
 'price',
 'our',
 'drugs',
 'the',
 'cost',
 'plus',
 'company',
 'will',
 'publish',
 'the',
 'costs',
 'to',
 'manufacture',
 'distribute',
 'and',
 'market',
 'its',
 'drugs',
 'to',
 'pharmacies',
 'it',
 'said',
 'it',
 'adds',
 'a',
 '15',
 'percent',
 'margin',
 'to',
 'get',
 'its',
 'wholesale',
 'prices',
 'and',
 'make',
 'sure',
 'it',
 'remains',
 'profitable',
 'but',
 'that',
 'there',
 'are',
 'no',
 'middlemen',
 'and',
 'no',
 'rebates',
 'for',
 'insurance',
 'companies',
 'everybody',
 'gets',
 'the',
 'same',
 'low',
 'pr

In [13]:
# Convert each word in y into their corresponding indices
y_ints = [word_indices[item] for item in y]
y_ints

[371,
 2993,
 3246,
 3141,
 2128,
 956,
 1651,
 1983,
 208,
 1554,
 1178,
 832,
 676,
 3291,
 2128,
 956,
 905,
 2549,
 1178,
 832,
 3291,
 2295,
 833,
 3589,
 2927,
 1888,
 1878,
 1022,
 3344,
 2624,
 2087,
 3509,
 2357,
 1690,
 1554,
 1179,
 371,
 2546,
 3344,
 2664,
 2703,
 3379,
 1785,
 1738,
 3584,
 2603,
 2419,
 1179,
 3291,
 905,
 2549,
 832,
 3621,
 2676,
 3291,
 908,
 3344,
 2120,
 1116,
 371,
 2129,
 1892,
 1179,
 3344,
 2515,
 1888,
 2910,
 1888,
 263,
 208,
 93,
 2493,
 2123,
 3344,
 1557,
 1892,
 3614,
 2604,
 371,
 2110,
 3221,
 1888,
 2784,
 2630,
 669,
 3288,
 3296,
 427,
 2311,
 2184,
 371,
 2311,
 2736,
 1478,
 1847,
 831,
 1300,
 1558,
 3291,
 2917,
 2086,
 2603,
 1478,
 1299,
 1178,
 3584,
 2110,
 3291,
 832,
 3589,
 3152,
 3291,
 1435,
 1178,
 1888,
 3621,
 2621,
 1878,
 320,
 365,
 398,
 1178,
 3291,
 832,
 1878,
 1726,
 3344,
 1861,
 2223,
 3284,
 79,
 1179,
 671,
 3291,
 1247,
 2357,
 126,
 442,
 3596,
 442,
 659,
 208,
 2514,
 1351,
 1785,
 976,
 671,
 127,
 32

In [14]:
# Create one-hot encoding of y
y_encoded = []

for item in y_ints:
    y_vector = np.zeros(len(unique_words))
    y_vector[item] = 1
    y_encoded.append(y_vector)

y_encoded = np.array(y_encoded)
y_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [15]:
class TextGenerator(nn.Module):
    def __init__(self, input_dim, output_dim, block_size):
        super().__init__()

        self.embeddings = nn.Linear(input_dim, 50 * 2)
        self.hidden = nn.Linear(50 * 2, 200)
        self.output = nn.Linear(200, output_dim)
        
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.sigmoid(self.embeddings(x))
        x = self.tanh(self.hidden(x))
        x = self.softmax(self.output(x))

        return x

In [16]:
# Get size of input for training the model
input_size = x_encoded[0].ravel().shape[0]
input_size

7386

In [17]:
# Instantiate model
model = TextGenerator(input_size, len(unique_words), block_size)

# Print model configuration
model

TextGenerator(
  (embeddings): Linear(in_features=7386, out_features=100, bias=True)
  (hidden): Linear(in_features=100, out_features=200, bias=True)
  (output): Linear(in_features=200, out_features=3693, bias=True)
  (tanh): Tanh()
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
)

In [18]:
# Create custom Dataset class
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        self.n_samples = len(x)
    
    def __getitem__(self, index):
        return self.x[index].ravel(), self.y[index]
    
    def __len__(self):
        return self.n_samples

In [19]:
# Create x and y PyTorch tensors
device = 'cpu'
x = torch.tensor(x_encoded).float().to(device)
y = torch.tensor(y_encoded).float().to(device)

In [20]:
# Create training dataset using custom Dataset class
training_ds = CustomDataset(x, y)

In [21]:
# Load training dataset into DataLoader
from torch.utils.data import DataLoader

batch_size = 5

train_loader = DataLoader(
    training_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

In [22]:
# Define model optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.000000001)
criterion = nn.CrossEntropyLoss()

In [23]:
# Define function to train model
def train_fn(loader, model, optimizer, loss_fn, device):
    loop = tqdm(loader)

    ave_loss = 0
    count = 0 
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Forward
        predictions = model.forward(data)
        loss = loss_fn(predictions, targets)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update tqdm loading bar
        loop.set_postfix(loss=loss.item())

        count += 1
        ave_loss += loss.item()
    
    ave_loss = ave_loss / count

    return ave_loss

In [24]:
def generate_text(model, seed_text, num_words):
    device = 'cpu'
    model.eval()
    
    # Convert seed_text to input tensor
    seed_encoded = []
    for word in seed_text.split():
        word_index = word_indices[word]
        word_encoded = np.zeros(len(unique_words))
        word_encoded[word_index] = 1
        seed_encoded.append(word_encoded)
    seed_encoded = np.array(seed_encoded)
    seed_encoded = np.expand_dims(seed_encoded, axis=0)
    seed_tensor = torch.tensor(seed_encoded).float().to(device)

    # Generate text
    generated_text = seed_text
    for i in range(num_words):
        predictions = model(seed_tensor)
        predicted_index = torch.argmax(predictions, dim=1).item()
        predicted_word = indices_words[predicted_index]
        generated_text += ' ' + predicted_word
        
        # Update seed tensor with predicted word
        predicted_encoded = np.zeros(len(unique_words))
        predicted_encoded[predicted_index] = 1
        predicted_encoded = np.expand_dims(predicted_encoded, axis=0)
        seed_tensor = torch.cat((seed_tensor[:, 1:, :], torch.tensor(predicted_encoded).float().to(device)), axis=1)

    return generated_text

In [25]:
# Train model
epochs = 300
average_losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    ave_loss = train_fn(train_loader, model, optimizer, criterion, device)
    
    print("Ave Loss: {}".format(ave_loss))
    average_losses.append(ave_loss)

Epoch: 0


 23%|███████████████▊                                                    | 595/2564 [00:09<00:32, 60.20it/s, loss=8.21]


KeyboardInterrupt: 

In [ ]:
# Inspect probability distribution of word tokens
import matplotlib.pyplot as plt
import pandas as pd

text = ['\n', '\n']
x_ints = [word_indices[item] for item in text]
x_vector = []

for item in x_ints:
    x_item = np.zeros(len(unique_words))
    x_item[item] = 1
    x_vector.append(x_item)

initial_input = torch.tensor([np.array([x_vector]).ravel()]).to(dtype=torch.float32)
output = model(initial_input)[0].detach().cpu().numpy()

df = pd.DataFrame(output)
df.plot.bar()
plt.yscale("log")
plt.show()

In [ ]:
# Generate text sample from model output
word_count = 100
text = []
paragraphs = 5

word1, word2 = "\n", "\n"

for p in range(paragraphs):
    for i in range(word_count):
        phrase = [word1, word2]
        x_ints = [word_indices[item] for item in phrase]
        x_vector = []

        for item in x_ints:
            x_item = np.zeros(len(unique_words))
            x_item[item] = 1
            x_vector.append(x_item)

        initial_input = torch.tensor([np.array([x_vector]).ravel()]).float()

        output = model(initial_input)[0].detach().cpu().numpy()

        # Workaround to fix occasional sum(pvals[:-1]) > 1.0  bug from implicit casting in np.random.multinomial 
        output = output.astype(float)
        output /= output.sum()

        index = np.where(np.random.multinomial(1, output) == 1)[0][0]
        word3 = indices_words[index]
        text.append(word3)

        # Use generated word from this run as seed for next run
        word1, word2 = word2, word3
    
    text.append(r'\n\n')

In [ ]:
print("Generated Text:")
text = ' '.join(text)
print(text)